<a href="https://colab.research.google.com/github/hss0727/BigData_project_Spark/blob/main/spark_ml_RF_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
import numpy as np
import pandas as pd
import sys
from sklearn.metrics import r2_score

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
!pip install findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 199 kB 50.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=c15ab34235ee96ba2aff9d759c249389ab7bb72945483430588d0e54d83d05bb
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [13]:
from pyspark.sql import SparkSession
spark = SparkSession \
  .builder \
  .appName("RF Regressor") \
  .getOrCreate()


In [14]:
spark

In [25]:
df_features = spark.read.csv("/content/drive/Shareddrives/502_project/ML/test_features.csv", header=True)

In [26]:
df_features.show()

+----------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|       _c0|         positive1|         negative2|          

## Clean data to prepare for machine learning
#### 1. Select 
#### 2. Create Target variables

## Test ML with Corn Futures data

In [68]:
# Select Columns
df_corn = df_features.select(df_fn.columns[0:-14])
df_corn.show(3)

+----------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|       _c0|         positive1|         negative2|               net3|   positive_change4|   negative_change5|        net_change6|         positive7|         negative8|               net9|  positive_change10| negative_change11|       net_change12|        sub_topic0|        sub_topic1|        sub_topic2|        sub_topic3|        sub_topic4|

In [69]:
# Drop Rows with NULL values in any columns
df_corn = df_corn.dropna()

In [70]:
df_corn.show(3)

+----------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+
|       _c0|         positive1|         negative2|               net3|   positive_change4|   negative_change5|        net_change6|         positive7|         negative8|               net9|  positive_change10|  negative_change11|       net_change12|        sub_topic0|        sub_topic1|        sub_topic2|        sub_topic3|        sub_topic4

### We can see that 2022-01-02 data dropped.
### Need to convert string fields to numeric type.

In [71]:
df_corn.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- positive1: string (nullable = true)
 |-- negative2: string (nullable = true)
 |-- net3: string (nullable = true)
 |-- positive_change4: string (nullable = true)
 |-- negative_change5: string (nullable = true)
 |-- net_change6: string (nullable = true)
 |-- positive7: string (nullable = true)
 |-- negative8: string (nullable = true)
 |-- net9: string (nullable = true)
 |-- positive_change10: string (nullable = true)
 |-- negative_change11: string (nullable = true)
 |-- net_change12: string (nullable = true)
 |-- sub_topic0: string (nullable = true)
 |-- sub_topic1: string (nullable = true)
 |-- sub_topic2: string (nullable = true)
 |-- sub_topic3: string (nullable = true)
 |-- sub_topic4: string (nullable = true)
 |-- sub_topic5: string (nullable = true)
 |-- sub_topic6: string (nullable = true)
 |-- sub_topic7: string (nullable = true)
 |-- sub_topic8: string (nullable = true)
 |-- sub_topic9: string (nullable = true)
 |-- sub_topic0_change:

In [75]:
# Change data types from string to FLOAT
df_final = df_corn.select("_c0", *(F.col(c).cast("float").alias(c) for c in df_corn.columns[1:]))

In [76]:
df_final.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- positive1: float (nullable = true)
 |-- negative2: float (nullable = true)
 |-- net3: float (nullable = true)
 |-- positive_change4: float (nullable = true)
 |-- negative_change5: float (nullable = true)
 |-- net_change6: float (nullable = true)
 |-- positive7: float (nullable = true)
 |-- negative8: float (nullable = true)
 |-- net9: float (nullable = true)
 |-- positive_change10: float (nullable = true)
 |-- negative_change11: float (nullable = true)
 |-- net_change12: float (nullable = true)
 |-- sub_topic0: float (nullable = true)
 |-- sub_topic1: float (nullable = true)
 |-- sub_topic2: float (nullable = true)
 |-- sub_topic3: float (nullable = true)
 |-- sub_topic4: float (nullable = true)
 |-- sub_topic5: float (nullable = true)
 |-- sub_topic6: float (nullable = true)
 |-- sub_topic7: float (nullable = true)
 |-- sub_topic8: float (nullable = true)
 |-- sub_topic9: float (nullable = true)
 |-- sub_topic0_change: float (nullable = tru

In [77]:
# Rename date column name
df_final = df_final.withColumnRenamed("_c0", "date")

In [78]:
df_final.show(3)

+----------+----------+----------+-----------+----------------+----------------+------------+---------+----------+-----------+-----------------+-----------------+------------+-----------+-----------+-----------+----------+-----------+------------+-----------+-----------+-----------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------+
|      date| positive1| negative2|       net3|positive_change4|negative_change5| net_change6|positive7| negative8|       net9|positive_change10|negative_change11|net_change12| sub_topic0| sub_topic1| sub_topic2|sub_topic3| sub_topic4|  sub_topic5| sub_topic6| sub_topic7| sub_topic8|sub_topic9|sub_topic0_change|sub_topic1_change|sub_topic2_change|sub_topic3_change|sub_topic4_change|sub_topic5_change|sub_topic6_change|sub_topic7_change|sub_topic8_change|sub_topic9_change|Corn Futures|
+----------+----------+-

### Train, Test split

In [ ]:
train_data, test_data, predict_data = df_final.randomSplit([0.8, 0.18, 0.02], 24)

In [137]:
train = df_final.filter(df_final.date <= "2022-07-31")
test = df_final.filter(df_final.date > "2022-07-31")

In [138]:
train.show(3)

+----------+----------+----------+-----------+----------------+----------------+------------+---------+----------+-----------+-----------------+-----------------+------------+-----------+-----------+-----------+----------+-----------+------------+-----------+-----------+-----------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------+
|      date| positive1| negative2|       net3|positive_change4|negative_change5| net_change6|positive7| negative8|       net9|positive_change10|negative_change11|net_change12| sub_topic0| sub_topic1| sub_topic2|sub_topic3| sub_topic4|  sub_topic5| sub_topic6| sub_topic7| sub_topic8|sub_topic9|sub_topic0_change|sub_topic1_change|sub_topic2_change|sub_topic3_change|sub_topic4_change|sub_topic5_change|sub_topic6_change|sub_topic7_change|sub_topic8_change|sub_topic9_change|Corn Futures|
+----------+----------+-

In [140]:
test.show(3)

+----------+----------+----------+------------+----------------+----------------+------------+----------+----------+-----------+-----------------+-----------------+------------+----------+-----------+------------+----------+-----------+-----------+-----------+-----------+-----------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------+
|      date| positive1| negative2|        net3|positive_change4|negative_change5| net_change6| positive7| negative8|       net9|positive_change10|negative_change11|net_change12|sub_topic0| sub_topic1|  sub_topic2|sub_topic3| sub_topic4| sub_topic5| sub_topic6| sub_topic7| sub_topic8|sub_topic9|sub_topic0_change|sub_topic1_change|sub_topic2_change|sub_topic3_change|sub_topic4_change|sub_topic5_change|sub_topic6_change|sub_topic7_change|sub_topic8_change|sub_topic9_change|Corn Futures|
+----------+----------

### Create Pipeline and train a model

In [130]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline, Model

In [102]:
cols = train.columns
cols[1:-1]

['positive1',
 'negative2',
 'net3',
 'positive_change4',
 'negative_change5',
 'net_change6',
 'positive7',
 'negative8',
 'net9',
 'positive_change10',
 'negative_change11',
 'net_change12',
 'sub_topic0',
 'sub_topic1',
 'sub_topic2',
 'sub_topic3',
 'sub_topic4',
 'sub_topic5',
 'sub_topic6',
 'sub_topic7',
 'sub_topic8',
 'sub_topic9',
 'sub_topic0_change',
 'sub_topic1_change',
 'sub_topic2_change',
 'sub_topic3_change',
 'sub_topic4_change',
 'sub_topic5_change',
 'sub_topic6_change',
 'sub_topic7_change',
 'sub_topic8_change',
 'sub_topic9_change']

In [108]:
# Create feature vector by combining all features together
vectorAssembler_features = VectorAssembler(
    inputCols= cols[1:-1], 
    outputCol= "features",
    handleInvalid = "skip")


In [109]:
# Define estimator
rfr = RandomForestRegressor(
    featuresCol= "features",
    labelCol= "Corn Futures"
)

In [110]:
pipeline_rfr = Pipeline(stages=[vectorAssembler_features,
                                rfr])

In [111]:
model_rfr = pipeline_rfr.fit(train)

In [113]:
pipelined_data = model_rfr.transform(train)

In [114]:
pipelined_data.show(3)

+----------+----------+----------+-----------+----------------+----------------+------------+---------+----------+-----------+-----------------+-----------------+------------+-----------+-----------+-----------+----------+-----------+------------+-----------+-----------+-----------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------+--------------------+--------------------+
|      date| positive1| negative2|       net3|positive_change4|negative_change5| net_change6|positive7| negative8|       net9|positive_change10|negative_change11|net_change12| sub_topic0| sub_topic1| sub_topic2|sub_topic3| sub_topic4|  sub_topic5| sub_topic6| sub_topic7| sub_topic8|sub_topic9|sub_topic0_change|sub_topic1_change|sub_topic2_change|sub_topic3_change|sub_topic4_change|sub_topic5_change|sub_topic6_change|sub_topic7_change|sub_topic8_change|sub_topic9_cha

### Model Test Results

In [121]:
predictions = model_rfr.transform(test)

In [133]:
evaluatorRF = RegressionEvaluator(labelCol= "Corn Futures", predictionCol="prediction", metricName="rmse")
rmse = evaluatorRF.evaluate(predictions)

In [135]:
rmse

0.020451203369531448

In [136]:
rfr_model = model_rfr.stages[1]
print(rfr_model)

RandomForestRegressionModel: uid=RandomForestRegressor_5e7885d30e82, numTrees=20, numFeatures=32
